In [1]:
SESSION_ID = "6d454e03-0ff6-4010-af7f-24b1ff46fd88"
TOKEN = "f3c19bf9-ca6c-4eea-b83e-d0b388ec4d2f"
BASE_URL = "https://zeus.accurate.id"
START_DATE = "01/01/2024"
END_DATE = "01/01/2026"

In [8]:
import requests
import pandas as pd
import time
import os
import datetime

In [4]:
all_invoices = []
page = 1
page_size = 100


while True:
    url = f"{BASE_URL}/accurate/api/sales-invoice/list.do"
    headers = {
        'Authorization': f'Bearer {TOKEN}',
        'X-Session-ID': SESSION_ID,
        'Content-Type': 'application/json'
    }

    params = {
        'sp.pageSize': page_size,
        'sp.page': page,
        'filter.transDate.op': 'BETWEEN',
        'filter.transDate.val[0]': START_DATE,
        'filter.transDate.val[1]': END_DATE
    }

    print(f"Fetching page {page}...")

    response = requests.post(url, headers=headers, params=params)
    data = response.json()

    invoices = data.get('d', [])

    print(f'success: {response.status_code}, found: {len(invoices)} invoices')

    if not invoices:
        break

    all_invoices.extend(invoices)
    print(f"Fetched page {page} with {len(invoices)} invoices.")

    page += 1
    time.sleep(1)  # Untuk menghindari rate limiting

print(f"Total invoices fetched: {len(all_invoices)}")

Fetching page 1...
success: 200, found: 100 invoices
Fetched page 1 with 100 invoices.
Fetching page 2...
success: 200, found: 100 invoices
Fetched page 2 with 100 invoices.
Fetching page 3...
success: 200, found: 100 invoices
Fetched page 3 with 100 invoices.
Fetching page 4...
success: 200, found: 100 invoices
Fetched page 4 with 100 invoices.
Fetching page 5...
success: 200, found: 100 invoices
Fetched page 5 with 100 invoices.
Fetching page 6...
success: 200, found: 100 invoices
Fetched page 6 with 100 invoices.
Fetching page 7...
success: 200, found: 100 invoices
Fetched page 7 with 100 invoices.
Fetching page 8...
success: 200, found: 100 invoices
Fetched page 8 with 100 invoices.
Fetching page 9...
success: 200, found: 100 invoices
Fetched page 9 with 100 invoices.
Fetching page 10...
success: 200, found: 100 invoices
Fetched page 10 with 100 invoices.
Fetching page 11...
success: 200, found: 100 invoices
Fetched page 11 with 100 invoices.
Fetching page 12...
success: 200, found

In [6]:
batch_counter = 0

for i, invoice in enumerate(all_invoices, 1):
    invoice_id = invoice.get('id')

    url = f"{BASE_URL}/accurate/api/sales-invoice/detail.do"
    headers = {
        'Authorization': f'Bearer {TOKEN}',
        'X-Session-ID': SESSION_ID,
        'Content-Type': 'application/json'
    }

    params = {
        'id': invoice_id
    }

    print(f"Fetching details for invoice {i}/{len(all_invoices)}: ID {invoice_id}...")

    response = requests.get(url, headers=headers, params=params)
    detail = response.json()

    all_invoices[i - 1] = detail.get('d', {})
    print(f'success: {response.status_code}, invoice ID: {invoice_id}')

    batch_counter += 1

    if batch_counter >= 5:
        batch_counter = 0
        time.sleep(1.5)  # Pause after every 5 requests

print("All invoice details fetched.")


Fetching details for invoice 1/5229: ID 99352...
success: 200, invoice ID: 99352
Fetching details for invoice 2/5229: ID 99204...
success: 200, invoice ID: 99204
Fetching details for invoice 3/5229: ID 99603...
success: 200, invoice ID: 99603
Fetching details for invoice 4/5229: ID 99602...
success: 200, invoice ID: 99602
Fetching details for invoice 5/5229: ID 99650...
success: 200, invoice ID: 99650
Fetching details for invoice 6/5229: ID 99551...
success: 200, invoice ID: 99551
Fetching details for invoice 7/5229: ID 99601...
success: 200, invoice ID: 99601
Fetching details for invoice 8/5229: ID 99053...
success: 200, invoice ID: 99053
Fetching details for invoice 9/5229: ID 99600...
success: 200, invoice ID: 99600
Fetching details for invoice 10/5229: ID 99550...
success: 200, invoice ID: 99550
Fetching details for invoice 11/5229: ID 99451...
success: 200, invoice ID: 99451
Fetching details for invoice 12/5229: ID 99500...
success: 200, invoice ID: 99500
Fetching details for invo

In [7]:
print(all_invoices[:2])  # Menampilkan 2 data pertama untuk verifikasi

[{'taxDateView': '31 Dec 2025', 'taxReference': None, 'printUserName': 'Management', 'reverseInvoice': False, 'journalDpId': None, 'numericField10': 0.0, 'expiredDate': '-', 'reverseInvoiceStatus': None, 'masterProjectId': None, 'receiveItem': False, 'lastReturnDate': None, 'currencyId': 50, 'efakturProceed': False, 'transDateView': '31 Dec 2025', 'tax1Id': None, 'branchId': 50, 'projectContractor': None, 'lastPaymentId': None, 'employeeLoanSettlement': False, 'billOfMaterial': False, 'taxNumber': None, 'retailWpName': 'PT. FOOD PACKAGING JAYA', 'tax1Amount': 0.0, 'salesOrder': False, 'commentCount': 0, 'hasNPWP': False, 'orderDownPayment': None, 'taxableAmount2': 0.0, 'taxableAmount1': 0.0, 'lastCashDiscPercent': '', 'taxableAmount4': 0.0, 'poNumber': 'FL202512004315', 'charField10': '', 'processStages': False, 'posCreateDate': None, 'preliminarySurvey': False, 'employeeLoan': False, 'availableDownPayment': 0.0, 'journalId': 125264, 'taxReturn': 0.0, 'customerHasProjectContractor': Fa

In [9]:
def calculate_age(date_str):
        """Calculate days from date to now"""
        if not date_str:
            return None
        
        try:
            dt = datetime.datetime.strptime(date_str, '%d/%m/%Y')
            age = (datetime.datetime.now() - dt).days
            return age
        except ValueError:
            return None

In [10]:
invoice_records = []
for inv in all_invoices:
    try:
        record = {
            # Primary Key
            'invoice_id': inv.get('id'),
            'invoice_number': inv.get('number'),
            
            # Dates
            'invoice_date': inv.get('transDate'),
            'due_date': inv.get('dueDate'),
            'ship_date': inv.get('shipDate'),
            
            # Customer
            'customer_id': inv.get('customerId'),
            'customer_name': inv.get('customer', {}).get('name') if inv.get('customer') else None,
            
            # Financial - Core
            'sub_total': float(inv.get('subTotal', 0)),
            'total_amount': float(inv.get('totalAmount', 0)),
            'outstanding_amount': float(inv.get('outstanding', 0)),
            
            # Status
            'status': inv.get('status'),
            'approval_status': inv.get('approvalStatus'),
            
            # Reference
            'po_number': inv.get('poNumber'),
            'sales_order_id': inv.get('salesOrderId'),
            'delivery_order_id': inv.get('deliveryOrderId'),
            
            # Payment Terms & Currency
            'payment_term_id': inv.get('paymentTermId'),
            'payment_term_name': inv.get('paymentTerm', {}).get('name') if inv.get('paymentTerm') else None,
            'currency_id': inv.get('currencyId'),
            'currency_code': inv.get('currency', {}).get('code') if inv.get('currency') else 'IDR',
            'exchange_rate': float(inv.get('rate', 1.0)),
            
            # Organization
            'branch_id': inv.get('branchId'),
            'branch_name': inv.get('branchName'),
            
            # Calculated
            'invoice_age_days': calculate_age(inv.get('transDate')),
            
            # Metadata
            'created_by': inv.get('createdBy'),
            'printed_time': inv.get('printedTime'),
            'extracted_at': datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
        
        invoice_records.append(record)
    except Exception as e:
        print(f"Error processing invoice ID {inv.get('id')}: {e}")
        continue
        
invoices_df = pd.DataFrame(invoice_records)
output_file = f"sales_invoices_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
invoices_df.to_csv(output_file, index=False)
print(f"Data saved to {output_file}")

Data saved to sales_invoices_20260203_221624.csv


In [12]:
detail_records = []
for inv in all_invoices:
    invoice_id = inv.get('id')
    
    for detail in inv.get('detailItem', []):
        try:
            # Get item info
            item = detail.get('item', {})
            item_unit = detail.get('itemUnit', {})
            
            record = {
                # Keys
                'detail_id': detail.get('id'),
                'invoice_id': invoice_id,
                'invoice_number': inv.get('number'),
                
                # Item
                'item_id': detail.get('itemId'),
                'item_number': item.get('no'),
                'item_name': item.get('name'),
                'item_category_id': item.get('itemCategoryId'),
                
                # Quantity & Unit
                'quantity': float(detail.get('quantity', 0)),
                'unit_id': detail.get('itemUnitId'),
                'unit_name': item_unit.get('name'),
                'unit_ratio': float(detail.get('unitRatio', 1.0)),
                
                # Pricing
                'unit_price': float(detail.get('unitPrice', 0)),
                'gross_amount': float(detail.get('grossAmount', 0)),
                'sales_amount': float(detail.get('salesAmount', 0)),
                
                # Warehouse
                'warehouse_id': detail.get('warehouseId'),
                'warehouse_name': detail.get('warehouse', {}).get('name') if detail.get('warehouse') else None,
                
                # Reference
                'sales_order_detail_id': detail.get('salesOrderDetailId'),
                'delivery_order_detail_id': detail.get('deliveryOrderDetailId'),
                
                # Sequence
                'line_seq': detail.get('seq'),
                
                # Metadata
                'extracted_at': datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }
            
            detail_records.append(record)
        except Exception as e:
            print(f"Error processing detail for invoice ID {invoice_id}: {e}")
            continue

details_df = pd.DataFrame(detail_records)
output_file_details = f"sales_invoice_details_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
details_df.to_csv(output_file_details, index=False)

Error processing detail for invoice ID 55752: 'NoneType' object has no attribute 'get'
Error processing detail for invoice ID 35250: 'NoneType' object has no attribute 'get'


In [ ]:
from nbformat import read
from nbconvert import PythonExporter

with open(notebook_path, "r", encoding="utf-8") as nb_file:
    notebook = read(nb_file, as_version=4)

exporter = PythonExporter()
script_body, _ = exporter.from_notebook_node(notebook)

with open(output_script, "w", encoding="utf-8") as script_file:
    script_file.write(script_body)

print(f"Script generated at {output_script}")